In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import pandas as pd
import os

In [2]:
csv_path = r"D:\Human Disease Diagnosis\dataset\Diabetes\train_1.csv" 
image_dir = r"D:\Human Disease Diagnosis\dataset\Diabetes\train"  
df = pd.read_csv(csv_path)

##### def preprocess_data(df, image_dir):
    df["image_path"] = df["id_code"].apply(lambda x: os.path.join(image_dir, x + ".png"))
    return df

In [4]:
df = preprocess_data(df, image_dir)

In [5]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [6]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1,
    shear_range=0.1
)

In [7]:
train_generator = datagen.flow_from_dataframe(
    df,
    x_col="image_path",
    y_col="diagnosis",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="raw",
    subset="training"
)


Found 826 validated image filenames.


C:\Users\Ribhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1898 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


In [8]:
val_generator = datagen.flow_from_dataframe(
    df,
    x_col="image_path",
    y_col="diagnosis",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="raw",
    subset="validation"
)

Found 206 validated image filenames.


In [9]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
out = Dense(5, activation="softmax")(x)  # 5 classes (0 to 4)

model = Model(inputs=base_model.input, outputs=out)

In [10]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [11]:
EPOCHS = 20
model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

C:\Users\Ribhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4342 - loss: 1.4367   

C:\Users\Ribhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 167s 6s/step - accuracy: 0.4343 - loss: 1.4356 - val_accuracy: 0.4660 - val_loss: 1.3006
Epoch 2/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 130s 5s/step - accuracy: 0.4992 - loss: 1.3208 - val_accuracy: 0.4660 - val_loss: 1.3070
Epoch 3/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 128s 5s/step - accuracy: 0.4841 - loss: 1.3124 - val_accuracy: 0.4660 - val_loss: 1.2818
Epoch 4/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 126s 5s/step - accuracy: 0.4860 - loss: 1.3212 - val_accuracy: 0.4660 - val_loss: 1.2950
Epoch 5/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 129s 5s/step - accuracy: 0.4485 - loss: 1.3727 - val_accuracy: 0.4660 - val_loss: 1.3116
Epoch 6/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 128s 5s/step - accuracy: 0.4930 - loss: 1.3373 - val_accuracy: 0.4660 - val_loss: 1.2935
Epoch 7/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 128s 5s/step - accuracy: 0.4835 - loss: 1.3594 - val_accuracy: 0.4660 - val_loss: 1.2802
Epoch 8/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 128s 5s/step - accuracy: 0.4608 - loss: 1.3664 - val_accuracy: 0.4660 - val_loss: 1.

In [12]:
model.save("diabetic_retinopathy_model.keras")